In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [16]:
from DataLoader import DataLoader as DL
import utils

In [11]:
import statsmodels.tsa.api as sta
import statsmodels.tsa.stattools as stattools
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import statsmodels.stats.stattools as stools

In [12]:
loc = 'C:/Users/Biao Feng/source/Anaconda3/practicum/'
d = DL(loc=loc)

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Biao Feng\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
atq = utils.get_time_series(d.merged_table, 'atq')
ltq = utils.get_time_series(d.merged_table, 'ltq')
niq = utils.get_time_series(d.merged_table, 'niq')
mcap = utils.get_time_series(d.merged_table, 'mcap')
ret = utils.get_time_series(d.merged_table, 'RET')

AttributeError: module 'utils' has no attribute 'get_time_series'

In [ ]:
d.merged_table.loc[:, 'in_universe']=[np.nan if pd.isnull(m) or pd.isnull(b) or m < b else 1 
                                      for m,b in zip(d.merged_table.mcap.values, d.merged_table.breakpt.values)]

In [ ]:
d.merged_table = d.industry_classifier(d.merged_table)

In [18]:
def get_time_series(df, col,icol='datadate'):

    
    if col not in df.columns:
        raise NameError("{col} does not exist".format(col=col))
    
    target_data =  df.loc[:, [icol, 'gvkey', col]]
    target_data = target_data.drop_duplicates([icol, 'gvkey', col], keep='first')
    #drop nan
    target_data = target_data[np.isnan(target_data[col]) == False]

    
    ## get multi data   
    tmp = target_data.groupby(['gvkey', 'datadate']).gvkey.count()
    tmp = tmp[tmp>1]
    tmp_index = tmp.index.values
    lala = target_data.reset_index(drop = True)
    multi = lala.loc[[0,1]]
    return multi

In [1]:
col = 'atq'
df = d.merged_table
icol='datadate'
target_data =  df.loc[:, [icol, 'gvkey', col]]
target_data = target_data.drop_duplicates([icol, 'gvkey', col], keep='first')
#drop nan
target_data = target_data[np.isnan(target_data[col]) == False]
#print(target_data)
tmp = target_data.groupby(['gvkey', 'datadate']).gvkey.count()
lala = target_data.reset_index(drop = True)
multi = lala.loc[[0,1]]
print(lala.loc[[0,1]])

NameError: name 'd' is not defined

In [65]:
import pandas as pd
import numpy as np
a = pd.DataFrame([[1,2,3],[2,3,4]], columns = [1,2,3])
print(a)
print(np.std(a))
print(a.div(a.std(1),0))
print(a.stack().reset_index().rename(columns={0:'book'}))

   1  2  3
0  1  2  3
1  2  3  4
1    0.5
2    0.5
3    0.5
dtype: float64
     1    2    3
0  1.0  2.0  3.0
1  2.0  3.0  4.0
   level_0  level_1  book
0        0        1     1
1        0        2     2
2        0        3     3
3        1        1     2
4        1        2     3
5        1        3     4


In [73]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
from DataLoader import DataLoader as DL

import statsmodels.tsa.api as sta
import statsmodels.tsa.stattools as stattools
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import statsmodels.stats.stattools as stools

import scipy.stats as stats

def norm_rank(kk):
    ## for every row, rank from small to big, then minus 0.5, 
    ## then divide number of data in row
    ## ppf: inverse function of cdf
    ## - 0.5 make sure no 1 or 0 in the ppf
    tmp = stats.norm.ppf((kk.rank(1)-.5).div(kk.count(1)))
    ## set index and columns
    kk = pd.DataFrame(tmp, index=kk.index, columns=kk.columns)
    ## normalization
    kk = kk.div(kk.std(1, ddof=0))
    return kk

loc='C:/Users/Biao Feng/source/Anaconda3/practicum/'
d = DL(loc)

##load all fundimental variables
for i in range(0,len(d.cols_q)):
    globals()[d.cols_q[i]] = d.get_time_series(d.merged_table, d.cols_q[i], icol='datadate')

mcap = d.get_time_series(d.merged_table, 'mcap')
ret = d.get_time_series(d.merged_table, 'RET')

## if data's mcap is bigger than breakpoint
d.merged_table.loc[:, 'in_universe']=[np.nan if pd.isnull(m) or pd.isnull(b) or m < b else 1 
                                      for m,b in zip(d.merged_table.mcap.values, d.merged_table.breakpt.values)]

# use data only after 1977
st = '1977'

#make example by industry 11
industry = 'i1'

#get industry universe
in_universe = d.get_time_series(d.merged_table[d.merged_table[industry]==1], 'in_universe')
in_univ = in_universe[st:].fillna(method='ffill', limit=3)
cols = in_univ.count()[in_univ.count()>20].index

##dependent variable
# book value growth
equity = (atq.loc[st:,cols]-ltq.loc[st:,cols]).rolling(20).mean()
equity_chg = np.log(equity).diff()
y_equchg = norm_rank(equity_chg).stack().reset_index().rename(columns={0: 'book_value growth'})
# book to market
b2m = np.log(equity / mcap * 1000)
y_b2m = norm_rank(b2m).stack().reset_index().rename(columns={0: 'book to market'})

##independent variable
for i in range(0,len(d.cols_q)):
    # 5 year mean
    tt = np.log(globals()[d.cols_q[i]].loc[st:,cols].rolling(20).mean())
    globals()['x_mean_' + d.cols_q[i]] = norm_rank(tt).stack().reset_index().rename(columns={0: d.cols_q[i]})
    # 5 year std
    ff = np.log(globals()[d.cols_q[i]].loc[st:,cols].rolling(20).std())
    globals()['x_std_' + d.cols_q[i]] = norm_rank(ff).stack().reset_index().rename(columns={0: d.cols_q[i]})

## get regression penal
reg_penal = pd.merge(y_equchg,y_b2m,on=['gvkey', 'datadate'], how ='inner')
for i in range(0,len(d.cols_q)):
    if len(globals()['x_mean_' + d.cols_q[i]]) > 2500:
        reg_penal = pd.merge(reg_penal, globals()['x_mean_' + d.cols_q[i]],
                             on=['gvkey', 'datadate'], how ='inner')
    if len(globals()['x_std_' + d.cols_q[i]]) > 2500:
        reg_penal = pd.merge(reg_penal, globals()['x_std_' + d.cols_q[i]],
                             on=['gvkey', 'datadate'], how ='inner')

AttributeError: 'DataLoader' object has no attribute 'cols_q'

In [75]:
def norm_rank(kk):
    ## for every row, rank from small to big, then minus 0.5, 
    ## then divide number of data in row
    ## ppf: inverse function of cdf
    ## - 0.5 make sure no 1 or 0 in the ppf
    tmp = stats.norm.ppf((kk.rank(1)-.5).div(kk.count(1)))
    ## set index and columns
    kk = pd.DataFrame(tmp, index=kk.index, columns=kk.columns)
    ## normalization
    kk = kk.div(kk.std(1, ddof=0))
    return kk


In [102]:
a = pd.DataFrame([[1,2,3]])
print(a.rank(1)-0.5)

     0
0  0.5
1  0.5
2  0.5


In [72]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = df.iloc[:, 4:].values
y = df['strength'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

slr = LinearRegression()

slr.fit(X_train, y_train)
y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)

NameError: name 'df' is not defined

In [70]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
X = reg_penal.iloc[:, 4:].values
y = reg_penal['book_value growth'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

NameError: name 'reg_penal' is not defined

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
data = y_train + X_train
model_name = ols('y_test ~ X_test',data = data).fit()

model_name.summary()
